In [9]:
import pandas as pd

from bengrn import get_GT_db, get_perturb_gt, get_sroy_gt, BenGRN, FILEDIR

import numpy as np
import tqdm
import scanpy as sc
import os

from huggingface_hub import hf_hub_download

from scprint import scPrint
from scprint.tasks import GNInfer
from scdataloader import Preprocessor

from bengrn import BenGRN
from anndata.utils import make_index_unique

from bengrn.base import train_classifier
from bengrn import compute_genie3
from grnndata import utils as grnutils
import joblib

from scdataloader.utils import revert_to_raw

import scipy.sparse

import torch

torch.set_float32_matmul_precision("medium")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## load gt

In [ ]:
gwps = get_perturb_gt()
gwps.var.index = gwps.var.gene_name
omni = get_GT_db("omnipath")
collectri = get_GT_db("collectri")

/home/ml4ig1/Documents code/benGRN/bengrn/base.py:657: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_sc.obs["organism_ontology_term_id"] = "NCBITaxon:9606"


In [ ]:
gwps = gwps.extract_links()

In [ ]:
# extracting PPI data from "computing the human interactome"
humaninter = pd.read_csv(
    "http://prodata.swmed.edu/humanPPI/bulk_download/GoodPairs.txt",
    sep="\t",
    skiprows=4,
)

In [ ]:
# getting U2OS cell map data
import json

try:
    import ndex2
except ImportError:
    os.system("uv pip install ndex2")
    import ndex2
from ndex2.cx2 import RawCX2NetworkFactory, CX2NetworkXFactory
from ndex2.cx2 import CX2NetworkPandasDataFrameFactory

# Create NDEx2 python client
client = ndex2.client.Ndex2()

# Create CX2Network factory
factory = RawCX2NetworkFactory()

# Download BioGRID: Protein-Protein Interactions (SARS-CoV) from NDEx
# https://www.ndexbio.org/viewer/networks/669f30a3-cee6-11ea-aaef-0ac135e8bacf
client_resp = client.get_network_as_cx2_stream("95bc75d5-d1d1-11ee-8a40-005056ae23aa")

# Convert downloaded network to CX2Network object
net_cx = factory.get_cx2network(json.loads(client_resp.content))

# Display information about network and output 1st 100 characters of CX2
print("Name: " + net_cx.get_name())
print("Number of nodes: " + str(len(net_cx.get_nodes())))
print("Number of nodes: " + str(len(net_cx.get_edges())))

# Creating an instance of CX2NetworkPandasDataFrameFactory
factory = CX2NetworkPandasDataFrameFactory()

# Converting CX2Network to DataFrame
cellmap = factory.get_dataframe(net_cx)


Name: U2OS AP-MS network
Number of nodes: 7543
Number of nodes: 36842


## load data

In [3]:
ls ./normalize_log

 one2one_orthologs/            task4_human.h5ad   task8_frog.h5ad
 README                        task4_mouse.h5ad   task9-1_ant.h5ad
'Supplementary Table 3.xlsx'   task6_nema.h5ad    task9-1_mouse2_adult.h5ad
 task36_sea_urchin.h5ad        task6_sty.h5ad     task9_ant.h5ad
 task36_zebrafish.h5ad         task7_cat.h5ad     task9_mouse.h5ad
 task3_cat.h5ad                task7_dog.h5ad
 task3_tiger.h5ad              task8_fish.h5ad


In [6]:
human = sc.read("./normalize_log/task4_human.h5ad")
mouse = sc.read("./normalize_log/task4_mouse.h5ad")

In [7]:
human.obs["organism_ontology_term_id"] = "NCBITaxon:9606"
mouse.obs["organism_ontology_term_id"] = "NCBITaxon:10090"

In [11]:
mouse = revert_to_raw(mouse)
human = revert_to_raw(human)

In [20]:
commonset = set(mouse.var.index.str.upper()) & set(human.var.index)

In [23]:
mouse.obs.tissue

AdultBladder_1.CCATCTAGCGAGTTTAGG    Bladder
AdultBladder_1.GAGGAGCGCTTGATACAG    Bladder
AdultBladder_1.CCAGACACAATAGAATTA    Bladder
AdultBladder_1.CCGACGGGACATATGGCG    Bladder
AdultBladder_1.TAGCATTCAAAGATTCCA    Bladder
                                      ...   
AdultMuscle_1.CCAGACATGCTTCGCACC      Muscle
AdultMuscle_1.TCGGGTACAATAAAGTAC      Muscle
AdultMuscle_1.TGCAATTCGTAACATCCC      Muscle
AdultMuscle_1.TCGGGTTTAACTGTCGGT      Muscle
AdultMuscle_1.AAAGTTGTAATGGCAGGA      Muscle
Name: tissue, Length: 31474, dtype: category
Categories (6, object): ['Bladder', 'Brain', 'Heart', 'Kidney', 'Liver', 'Muscle']

In [30]:
mouse.obs.celltype.unique().tolist()

['Epithelial cell',
 'Dendritic cell',
 'Macrophage',
 'Monocyte',
 'Basal cell',
 'T cell',
 'Fibroblast',
 'Neutrophil',
 'Smooth muscle cell',
 'Endothelial cell',
 'Erythroid cell',
 'B cell',
 'Gastric epithelial cell',
 'Unknown',
 'Secretory alveoli cell',
 'Myocyte',
 'Ependymal cell',
 'Glia',
 'Astrocyte',
 'Oligodendrocyte',
 'Microglia',
 'Mesenchymal cell',
 'Plasma cell',
 'Mast cell',
 'Neural progenitor cell',
 'Cardiomyocyte',
 'Proliferating cell',
 'Adipocyte cell',
 'Renal epithelial cell',
 'Chondrocyte',
 'Erythroblast',
 'Megakaryocyte',
 'Proximal tubule cell',
 'Intercalated cells of collecting duct',
 'Ascending loop of Henle',
 'Connecting tubule cell',
 'Striated muscle cell',
 'Distal convoluted tubule',
 'Hepatocyte',
 'Enterocyte',
 'Goblet cell']

In [ ]:
macr

In [ ]:
"Fibroblast" | "Macrophage"

In [ ]:
"stromal cell", "fetal stromal"

In [ ]:
'B cell', "T cell"

In [27]:
human.obs.celltype.unique().tolist()

['Stromal cell',
 'Fibroblast',
 'Endothelial cell (APC)',
 'Smooth muscle cell',
 'Epithelial cell (intermediated)',
 'B cell (Plasmocyte)',
 'Macrophage',
 'Endothelial cell',
 'Endothelial cell (endothelial to mesenchymal transition)',
 'Mast cell',
 'Monocyte',
 'Dendritic cell',
 'Intermediated cell',
 'T cell',
 'Gastric endocrine cell',
 'Basal cell',
 'Fetal fibroblast',
 'Epithelial cell',
 'Ventricle cardiomyocyte',
 'Antigen presenting cell (RPS high)',
 'M2 Macrophage',
 'Sinusoidal endothelial cell',
 'Neutrophil',
 'B cell',
 'Fasciculata cell',
 'Myeloid cell',
 'Intercalated cell',
 'Loop of Henle',
 'Erythroid progenitor cell (RP high)',
 'Fetal epithelial progenitor',
 'Ureteric bud cell',
 'hESC',
 'Stratified epithelial cell',
 'Proximal tubule progenitor',
 'Fetal enterocyte ',
 'Proliferating T cell',
 'CB CD34+',
 'Fetal stromal cell',
 'Fetal mesenchymal progenitor',
 'Neutrophil (RPS high)',
 'Pancreas exocrine cell',
 'Enterocyte progenitor',
 'Goblet cell',
 

In [22]:
human.obs.tissue

AdultBladder_1.CTCGCACAACAAGCTGTG-0    Bladder
AdultBladder_1.CTCGCAATTTGCGGTACA-0    Bladder
AdultBladder_1.AGGACTCGGCAGATTTGC-0    Bladder
AdultBladder_1.ACAATACGGCAGCGGCAG-0    Bladder
AdultBladder_1.AAGCGGAAAACGCGCACC-0    Bladder
                                        ...   
AdultKidney_4.TAGTCGTAAGGGGACACT2-1     Kidney
AdultKidney_4.TTCATACGAGTATCGGGT2-1     Kidney
AdultKidney_4.ACGTTGTAGTCGAACGCC2-1     Kidney
AdultKidney_4.AACGCCAAAACGTGCAAT2-1     Kidney
AdultKidney_4.AAGTACCCGACGGGGTTT2-1     Kidney
Name: tissue, Length: 52816, dtype: category
Categories (5, object): ['Bladder', 'Heart', 'Kidney', 'Liver', 'Muscle']

In [12]:
human.X = scipy.sparse.csr_matrix(human.X)
mouse.X = scipy.sparse.csr_matrix(mouse.X)

In [ ]:
preprocessor = Preprocessor(
    is_symbol=True, force_preprocess=True, skip_validate=True, do_postp=False
)
human_p = preprocessor(human.copy())
mouse_p = preprocessor(mouse.copy())

Dropping layers:  KeysView(Layers with keys: )
checking raw counts
removed 0 non primary cells, 52816 renamining
filtered out 5 cells, 52811 renamining
Removed 4336 genes not known to the ontology
Removed 0 duplicate genes
Added 53976 genes in the ontology but not present in the dataset
starting QC
Seeing 5689 outliers (10.77% of total dataset):
done
AnnData object with n_obs × n_vars = 52811 × 70611
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'tissue', 'n_genes', 'n_counts', 'sample', 'stage', 'celltype', 'batch', 'organism_ontology_term_id', 'nnz', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier'
    var: 'n_cells', 'uid', 'symbol', 'biotype', 'organism_id', 'branch_id', 'mt', 'ribo', 'hb', 'organism

## generate results

In [ ]:
model_checkpoint_file = "../../../w937u4o1.ckpt"  # w937u4o1.ckpt"


In [ ]:
model = scPrint.load_from_checkpoint(model_checkpoint_file, precpt_gene_emb=None)
model = model.to("cuda")

Gene position encoding has changed in the dataloader compared to last time, trying to revert
FYI: scPrint is not attached to a `Trainer`.


In [ ]:
CELLTYPES = ["oligodendrocyte"]

NUM_GENES = 5000
MAXCELLS = 1024
LOC = "../..data/temp/"  #

In [ ]:
# only dataset dropped
adata = sc.read(
    LOC + "glio_smart_highdepth" + ".h5ad",
    backup_url="https://datasets.cellxgene.cziscience.com/6ec440b4-542a-4022-ac01-56f812e25593.h5ad",
)

adata.var["isTF"] = False
adata.var.loc[adata.var.feature_name.isin(grnutils.TF), "isTF"] = True
adata

AnnData object with n_obs × n_vars = 3589 × 22149
    obs: 'Sample.type', 'Selection', 'Location', 'Sample.name', 'Total_reads', 'Unique_reads', 'Unique_reads_percent', 'Splice_sites_total', 'Splice_sites_Annotated', 'Splice_sites_GT.AG', 'Splice_sites_GC.AG', 'Splice_sites_AT.AC', 'Splice_sites_non_canonical', 'Multimapping_reads_percent', 'Unmapped_mismatch', 'Unmapped_short', 'Unmapped_other', 'ERCC_reads', 'Non_ERCC_reads', 'ERCC_to_non_ERCC', 'Genes_detected', 'housekeeping_cluster', 'Cluster_2d', 'Cluster_2d_color', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'donor_id', 'suspension_type', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered'

In [ ]:
adata.obs.cell_type.value_counts()

cell_type
myeloid cell                      1842
neoplastic cell                   1079
oligodendrocyte precursor cell     406
astrocyte                           88
oligodendrocyte                     81
vascular lymphangioblast            50
neuron                              21
Name: count, dtype: int64

In [ ]:
preprocessor = Preprocessor(force_preprocess=True, skip_validate=True, do_postp=False)
adata = preprocessor(adata.copy())

Dropping layers:  KeysView(Layers with keys: )
checking raw counts
removed 0 non primary cells, 3589 renamining
filtered out 22 cells, 3567 renamining
Removed 7 genes not known to the ontology
Removed 0 duplicate genes
Added 48469 genes in the ontology but not present in the dataset
starting QC
Seeing 71 outliers (1.99% of total dataset):
done
AnnData object with n_obs × n_vars = 3567 × 70611
    obs: 'Sample.type', 'Selection', 'Location', 'Sample.name', 'Total_reads', 'Unique_reads', 'Unique_reads_percent', 'Splice_sites_total', 'Splice_sites_Annotated', 'Splice_sites_GT.AG', 'Splice_sites_GC.AG', 'Splice_sites_AT.AC', 'Splice_sites_non_canonical', 'Multimapping_reads_percent', 'Unmapped_mismatch', 'Unmapped_short', 'Unmapped_other', 'ERCC_reads', 'Non_ERCC_reads', 'ERCC_to_non_ERCC', 'Genes_detected', 'housekeeping_cluster', 'Cluster_2d', 'Cluster_2d_color', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_

In [ ]:
metrics = {}
clf_omni = None
shapes = []
grn_inferer = GNInfer(
    how="most var within",
    preprocess="softmax",
    head_agg="mean",
    filtration="none",
    num_genes=2000,
    max_cells=256,
    num_workers=8,
    batch_size=2,
)
grn = grn_inferer(model, adata[(adata.X.sum(1) > 500)])
grn.var.index = make_index_unique(grn.var["symbol"].astype(str))
# grn.varp["all"] = grn.varp["GRN"]
# grn.varp["GRN"] = grn.varp["GRN"][:, :, :].mean(-1)
# metrics[celltype + "_scprint_mean"] = BenGRN(grn).scprint_benchmark()

number of expressed genes in this cell type: 20704
not on wandb, could not set name


100%|██████████| 128/128 [00:07<00:00, 16.07it/s]
... storing 'biotype' as categorical
... storing 'organism' as categorical


avg link count: 3549456, sparsity: 1.0


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

## bench variations